In [2]:
import json
import pandas as pd
import numpy as np

In [13]:
with open('output/levenshtein.json','r') as out:
    for line in out:
        data_1 = json.loads(line)

In [14]:
with open('output/levenshtein_soundex.json','r') as out:
    for line in out:
        data_2 = json.loads(line)

In [15]:
with open('output/levenshtein_metaphone.json','r') as out:
    for line in out:
        data_3 = json.loads(line)

In [16]:
levenshtein = pd.DataFrame(data_1)
levenshtein_soundex = pd.DataFrame(data_2)
levenshtein_metaphone = pd.DataFrame(data_3)

In [48]:
levenshtein.is_correct.value_counts()

True     582
False    219
Name: is_correct, dtype: int64

In [49]:
levenshtein_soundex.is_correct.value_counts()

True     582
False    219
Name: is_correct, dtype: int64

In [50]:
levenshtein_metaphone.is_correct.value_counts()

True     577
False    224
Name: is_correct, dtype: int64

In [33]:
levenshtein_metaphone_true = [d for d in data_3 if d['is_correct'] == True]
levenshtein_metaphone_false = [d for d in data_3 if d['is_correct'] == False]
print(len(levenshtein_metaphone_true), len(levenshtein_metaphone_false))

(577, 224)


In [66]:
#levenshtein.candidates.apply(lambda x: len(x)).average()
levenshtein['candidate_len'] = levenshtein.candidates.str.len()
levenshtein.candidate_len.mean()

1.2883895131086143

In [67]:
levenshtein_soundex['candidate_len'] = levenshtein_soundex.candidates.str.len()
levenshtein_soundex.candidate_len.mean()

1.0524344569288389

In [68]:
levenshtein_metaphone['candidate_len'] = levenshtein_metaphone.candidates.str.len()
levenshtein_metaphone.candidate_len.mean()

0.97128589263420728

In [72]:
display(levenshtein)
display(levenshtein_soundex)

,best_match,candidates,canonical,is_correct,token,candidate_len
0,new,[new],new,True,new,1
1,pix,[pix],pictures,False,pix,1
2,coming,"[coming, commaing, coaming, combing, cumming, ...",coming,True,comming,7
3,,[],tomorrow,False,tomoroe,0
4,dang,[dang],dang,True,dang,1
5,,[],fucking,False,effin,0
6,greenway,"[greenway, greenaway, greenbank, greeny, green...",greenbay,False,greenbay,6
7,bullshit,"[bullshit, bullit]",bullshit,True,bullshit,2
8,calls,"[calls, callas, calles]",calls,True,calls,3
9,nd,[nd],and,False,nd,1


,best_match,candidates,canonical,is_correct,token,candidate_len
0,new,[new],new,True,new,1
1,pix,[pix],pictures,False,pix,1
2,coming,"[coming, commaing, coaming, cumming]",coming,True,comming,4
3,,[],tomorrow,False,tomoroe,0
4,dang,[dang],dang,True,dang,1
5,,[],fucking,False,effin,0
6,greenbank,"[greenbank, greenbark, greenback]",greenbay,False,greenbay,3
7,bullshit,[bullshit],bullshit,True,bullshit,1
8,calls,"[calls, calles, callas]",calls,True,calls,3
9,nd,[nd],and,False,nd,1
